In [1]:
import pandas as pd
import numpy as np
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score ,f1_score,roc_curve
import matplotlib.pyplot as plt

from bert import BertModelLayer

l_bert = BertModelLayer(**BertModelLayer.Params(
  vocab_size               = 16000,        # embedding params
  use_token_type           = True,
  use_position_embeddings  = True,
  token_type_vocab_size    = 2,

  num_layers               = 12,           # transformer encoder params
  hidden_size              = 768,
  hidden_dropout           = 0.1,
  intermediate_size        = 4*768,
  intermediate_activation  = "gelu",

  adapter_size             = None,         # see arXiv:1902.00751 (adapter-BERT)

  shared_layer             = False,        # True for ALBERT (arXiv:1909.11942)
  embedding_size           = None,         # None for BERT, wordpiece embedding size for ALBERT

  name                     = "bert"        # any other Keras layer params
))

In [2]:
import bert

model_dir = "uncased_L-12_H-768_A-12"

bert_params = bert.params_from_pretrained_ckpt(model_dir)
l_bert = bert.BertModelLayer.from_params(bert_params, name="bert")

In [3]:
from tensorflow import keras

max_seq_len = 128
l_input_ids      = keras.layers.Input(shape=(max_seq_len,), dtype='int32')
l_token_type_ids = keras.layers.Input(shape=(max_seq_len,), dtype='int32')

# using the default token_type/segment id 0
output = l_bert(l_input_ids)                              # output: [batch_size, max_seq_len, hidden_size]
model = keras.Model(inputs=l_input_ids, outputs=output)
model.build(input_shape=(None, max_seq_len))


In [4]:
%load_ext autoreload
%autoreload 2
from data_prep import df_prep  
from data_prep import  NLP_Vectorizer
from data_prep import parse_line
from model_src import NLP_model


[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/nathan/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/nathan/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/nathan/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/nathan/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/nathan/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /Users/nathan/nltk_data...
[nltk_data]    |   Package movie_reviews is already

In [5]:
col_names = ['marketplace','customer_id','review_id','product_id','product_parent','product_title','product_category','star_rating','helpful_votes','total_votes','vine','verified_purchase','review_headline','review_body','review_date']
cols = {}
for i in range(len(col_names)):
    print (str(i)+': '+col_names[i])
    cols[col_names[i]] = i

0: marketplace
1: customer_id
2: review_id
3: product_id
4: product_parent
5: product_title
6: product_category
7: star_rating
8: helpful_votes
9: total_votes
10: vine
11: verified_purchase
12: review_headline
13: review_body
14: review_date


In [6]:
np.random.seed(500)
df = pd.read_csv('data/sample_02.csv')
#df = df[df['7']==1]
df = df[df['9']>10]

df = df.sample(frac=.2, random_state=1)
len(df)

3846

In [7]:
#Corpus = df_prep(df,np.mean(df['8']/df['9']),.0)
Corpus = pd.read_csv('data/Books_02_Corpus.csv')
Corpus['text_final'] = Corpus['text_final'].fillna(' ')
Corpus  = Corpus.sample(frac=.33, random_state=1)

In [8]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus[['text','text_final','help_score','help_votes','stars']],Corpus['label'],test_size=0.3)




In [9]:
TF_IDF_2K =NLP_Vectorizer('TF_IDF',{'max_features':2000,'ngram_range':(2,3)})

In [10]:
orig_Train_Y = Train_Y
Test_Y = TF_IDF_2K.encode_Y(Test_Y)
Train_Y = TF_IDF_2K.encode_Y(Train_Y)

# Begin Bert

In [11]:
train_df_bert = pd.DataFrame({
    'id':range(len(Train_X)),
    'label':Train_Y,
    'alpha':['a']*Train_X.shape[0],
    'text': Train_X['text_final'].replace(r'\n', ' ', regex=True)
})

train_df_bert.head()

,id,label,alpha,text
10214,0,0,a,year ago european nation dominate world absolu...
13263,1,0,a,harry turtledove poor writer concept alternate...
31270,2,1,a,idea author try jump hb nostalgia bandwagon be...
23610,3,0,a,little learning dangerous thing say alexander ...
21867,4,0,a,really excite try bette book hear many fantast...


In [12]:
dev_df_bert = pd.DataFrame({
    'id':range(len(Test_X)),
    'label':Test_Y,
    'alpha':['a']*Test_X.shape[0],
    'text': Test_X['text_final'].replace(r'\n', ' ', regex=True)
})

train_df_bert.head()

,id,label,alpha,text
10214,0,0,a,year ago european nation dominate world absolu...
13263,1,0,a,harry turtledove poor writer concept alternate...
31270,2,1,a,idea author try jump hb nostalgia bandwagon be...
23610,3,0,a,little learning dangerous thing say alexander ...
21867,4,0,a,really excite try bette book hear many fantast...


In [13]:

train_df_bert.to_csv('data/train.tsv', sep='\t', index=False, header=False)

In [14]:
dev_df_bert.to_csv('data/dev.tsv', sep='\t', index=False, header=False)

In [15]:
import torch
import pickle
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from torch.nn import CrossEntropyLoss, MSELoss

from tqdm import tqdm_notebook, trange
import os
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForSequenceClassification
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule

from multiprocessing import Pool, cpu_count
from tools import *
import convert_examples_to_features

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
processor = BinaryClassificationProcessor()
train_examples = processor.get_train_examples(DATA_DIR)
train_examples_len = len(train_examples)

NameError: name 'DATA_DIR' is not defined

In [ ]:
label_list = processor.get_labels() # [0, 1] for binary classification
num_labels = len(label_list)

In [ ]:
num_train_optimization_steps = int(
    train_examples_len / TRAIN_BATCH_SIZE / GRADIENT_ACCUMULATION_STEPS) * NUM_TRAIN_EPOCHS

In [ ]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

In [ ]:

label_map = {label: i for i, label in enumerate(label_list)}
train_examples_for_processing = [(example, label_map, MAX_SEQ_LENGTH, tokenizer, OUTPUT_MODE) for example in train_examples]

In [ ]:
process_count = cpu_count() - 1
if __name__ ==  '__main__':
    print(f'Preparing to convert {train_examples_len} examples..')
    print(f'Spawning {process_count} processes..')
    with Pool(process_count) as p:
        train_features = list(tqdm_notebook(p.imap(convert_examples_to_features.convert_example_to_feature, train_examples_for_processing), total=train_examples_len))

In [ ]:
with open(DATA_DIR + "train_features.pkl", "wb") as f:
    pickle.dump(train_features, f)

In [ ]:
# Load pre-trained model (weights)
model = BertForSequenceClassification.from_pretrained(BERT_MODEL, cache_dir=CACHE_DIR, num_labels=num_labels)
# model = BertForSequenceClassification.from_pretrained(CACHE_DIR + 'cased_base_bert_pytorch.tar.gz', cache_dir=CACHE_DIR, num_labels=num_labels)

In [ ]:
model.to(device)